A Python notebook that demonstrates, documents and benchmarks the value of using Apache Arrow to read in a large (5Gb) CSV file of years of historic environmental monitoring sample data (available from https://pub.data.gov.bc.ca/datasets/949f2233-9612-4b06-92a9-903e817da659/ems_sample_results_historic_expanded.zip) and compute the mean Result from all measurements of the Parameter “Chromium Total” where the units were recorded in mg/L. 

- working solution reads file and computes correct mean,
- provides indication of file sizes, read and compute times – with and without Arrow,
- solution briefly explains Apache Arrow and documents value.

## First, we get the time to read csv file and compute mean of Result without Apach Arrow

Get the time to read the CSV file.

In [1]:
import time
import os
import pandas as pd

csv_file = '/Users/raysu/Downloads/ems_sample_results_historic_expanded.csv'

# Display the size of the file
file_size = os.path.getsize(csv_file)
print("The CSV file size is {:,d} bytes".format(file_size))

# Run the test for a couple times and print the average time
read_times = []
for i in range(2):
    file_read_start = time.time()
    ems_df = pd.read_csv(csv_file)
    file_read_end = time.time()
    read_times.append(file_read_end - file_read_start)
    
file_read_time = sum(read_times)/len(read_times)

print("Reading CSV file '{}' took {} seconds.".format(os.path.basename(csv_file), file_read_time))

The CSV file size is 5,322,234,734 bytes


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (30,33,42,43,44,45,47,49,52,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Reading CSV file 'ems_sample_results_historic_expanded.csv' took 213.4216649532318 seconds.


Get the time to compute mean for all Chromium Total.

In [2]:
compute_times = []
result_means = set()

for i in range(3):
    compute_start = time.time()
    ct_df = ems_df[["EMS_ID", 'PARAMETER', 'RESULT']]
    chromium_total_df = ct_df.loc[ct_df['PARAMETER'] == 'Chromium Total']
    result_means.add(chromium_total_df['RESULT'].mean())
    compute_end = time.time()
    compute_times.append(compute_end - compute_start)
    
mean_compute_time = sum(compute_times)/len(compute_times)
if len(result_means) == 1:
    ct_result_mean = list(result_means)[0]
else:
    print("Unexpected things happened when calculating the mean of the Result.")

print("Computing the mean of 'Chromium Total' took {} seonds, and the mean is {}.".format(mean_compute_time, ct_result_mean))
chromium_total_df.head()

Computing the mean of 'Chromium Total' took 1.9945855935414631 seonds, and the mean is 1.5669242911547947.


,EMS_ID,PARAMETER,RESULT
603,0190039,Chromium Total,0.02
701,0131140,Chromium Total,0.01
875,E206114,Chromium Total,0.02
1100,0301315,Chromium Total,0.05
1779,0900503,Chromium Total,0.01


## Now we try it with Apache Arrow

In [3]:
from pyarrow import csv

csv_file = '/Users/raysu/Downloads/ems_sample_results_historic_expanded.csv'

# Display the size of the file
file_size = os.path.getsize(csv_file)
print("The CSV file size is {:,d} bytes".format(file_size))

print("Start reading EMS sample result history with Apache Arrow...")

# Run the test for 3 times and print the average time
read_times = []
for i in range(2):
    file_read_start = time.time()
    ems_table = csv.read_csv(csv_file)
    file_read_end = time.time()
    read_times.append(file_read_end - file_read_start)
    
file_read_time = sum(read_times)/len(read_times)

print("Reading CSV file '{}' took {} seconds with Apache Arrow.".format(os.path.basename(csv_file), file_read_time))

The CSV file size is 5,322,234,734 bytes
Start reading EMS sample result history with Apache Arrow...
Reading CSV file 'ems_sample_results_historic_expanded.csv' took 302.7074272632599 seconds with Apache Arrow.


In [4]:
import pyarrow.compute as pc

compute_times = []
result_means = set()

for i in range(3):
    compute_start = time.time()
    ems_df = ems_table.select(['EMS_ID', 'PARAMETER', 'RESULT']).to_pandas()
    ct_result_df = ems_df.loc[ems_df['PARAMETER'] == 'Chromium Total']
    result_means.add(ct_result_df['RESULT'].mean())
    compute_end = time.time()
    compute_times.append(compute_end - compute_start)
    
mean_compute_time = sum(compute_times)/len(compute_times)
if len(result_means) == 1:
    ct_result_mean = list(result_means)[0]
else:
    print("Unexpected things happened when calculating the mean of the Result.")

print("Computing the mean of 'Chromium Total' took {} seonds, and the mean is {}.".format(mean_compute_time, ct_result_mean))

Computing the mean of 'Chromium Total' took 41.501238425572716 seonds, and the mean is 1.5669242911547947.


- The CSV file used in the test has the size of 5,322,234,734 bytes.
- Without Apache Arrow, reading the file contents into a Pandas dataframe takes ~213.42 seconds. With Apache Arrow, it  reads in the same file to a pyarrow table in ~302.71 seconds, which is roughly 41.84% slower.
- Without Apache Arrow, computing the mean Result of the Paramenter 'Chronium Total' takes ~1.99 seconds. With Apache Arrow, it computes the mean in ~41.5 seconds, which is ~1985.43% slower. 
- The mean Result is 1.5669242911547947 mg/L.

Conclusion:
Apache Arrow does **not** seem to improve performance of reading large CSV files or the compute of Result mean. In fact, the compute time is a lot worse than without it. This may be because Apache Arrow does not support query table data at the moment. As a result, we have to convert pyarrow table to pandas dataframe before doing the calculation. Note: I did not explore iterating pyarrow table rows and do the calculation that way (seems like looping through rows is not supported.) It may worth adding a couple lines to measure the pyarrow Table to Pandas DataFrame conversion.

Note: The Jupyter Notebook was running on my laptop. If running on a well resourced server, the iteration may be increased to get a better evaluation.